In [3]:
from newsapi import NewsApiClient
import pandas as pd
from datetime import date, timedelta
from newspaper import Article, Config
import nltk
from nltk.corpus import stopwords

today = date.today()
yesterday = today - timedelta(days = 1)
day_before_yesterday = today - timedelta(days = 2)

# Init
newsapi = NewsApiClient(api_key='ff4373852c2343a98303951439854f8c')

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(   
                                          category='entertainment',
                                          language='en',
                                          page_size = 90,
                                          page= 1)

articles = top_headlines.get('articles',[])

init_df = pd.DataFrame(articles, columns = ['source','title','publishedAt','author','url'])

init_df['source'] = init_df['source'].apply(lambda x: x['name'] if pd.notna(x) and 'name' in x else None)

init_df['publishedAt'] = pd.to_datetime(init_df['publishedAt'])



filtered_df = init_df[(init_df['publishedAt'].dt.date == day_before_yesterday) | (init_df['publishedAt'].dt.date == yesterday)]

df = filtered_df.copy()

def full_content(url):
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
    config = Config()
    config.browser_user_agent = user_agent
    page = Article(url, config = config)

    try:
        page.download()
        page.parse()
        return page.text
    except Exception as e:
        print(f"Error retrieving content from {url}: {e}")
        return 'couldnt retrieve'


df['content'] = df['url'].apply(full_content)
df['content'] = df['content'].str.replace('\n', ' ')
df = df[df['content'] != 'couldnt retrieve']



# Download the stopwords dataset
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


def count_words_without_stopwords(text):
    if isinstance(text, (str, bytes)):
        words = nltk.word_tokenize(str(text))
        stop_words = set(stopwords.words('english'))
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return len(filtered_words)
    else:
        0

df['word_count'] = df['content'].apply(count_words_without_stopwords)
df


Error retrieving content from https://www.usatoday.com/story/entertainment/celebrities/2024/04/04/molly-ringwald-daughter-mathilda-gianopoulos-conceived-studio-54/73208054007/: Article `download()` failed with 403 Client Error: Unknown Error for url: https://eu.usatoday.com/story/entertainment/celebrities/2024/04/04/molly-ringwald-daughter-mathilda-gianopoulos-conceived-studio-54/73208054007/ on URL https://www.usatoday.com/story/entertainment/celebrities/2024/04/04/molly-ringwald-daughter-mathilda-gianopoulos-conceived-studio-54/73208054007/
Error retrieving content from https://www.washingtonpost.com/entertainment/tv/2024/04/04/carrie-preston-elsbeth-profile/: Article `download()` failed with HTTPSConnectionPool(host='www.washingtonpost.com', port=443): Read timed out. (read timeout=7) on URL https://www.washingtonpost.com/entertainment/tv/2024/04/04/carrie-preston-elsbeth-profile/
Error retrieving content from https://www.washingtonpost.com/books/2024/04/04/becky-lynch-wwe-wrestlema

[nltk_data] Downloading package stopwords to /home/gitpod/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/gitpod/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/gitpod/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,source,title,publishedAt,author,url,content,word_count
0,Entertainment Weekly,9-1-1's Oliver Stark breaks down Buck's kiss w...,2024-04-05 01:01:13+00:00,https://www.facebook.com/entertainmentweekly,https://ew.com/911-buck-tommy-kiss-bisexual-ol...,While the end of Thursday's 9-1-1 may have giv...,762
1,Fox News,"Madonna fires back at concert lawsuit, claims ...",2024-04-05 00:58:00+00:00,Brie Stimson,https://www.foxnews.com/entertainment/madonna-...,Madonna has fired back at a lawsuit brought by...,444
2,Vulture,"'Ripley' Recap, Episode 6: Some Heavy Instrume...",2024-04-05 00:48:06+00:00,Rafaela Bassili,http://www.vulture.com/article/ripley-recap-ep...,"As we open on “Some Heavy Instrument,” the six...",1199
3,Fox Business,Rebel Wilson claims agency 'liked me fat' afte...,2024-04-05 00:42:00+00:00,Janelle Ash,https://www.foxbusiness.com/entertainment/rebe...,Rebel Wilson reportedly secured a $10 million ...,396
4,Rolling Stone,"Kiss Sell Music Catalog, Publishing, and Image...",2024-04-05 00:35:00+00:00,Daniel Kreps,http://www.rollingstone.com/music/music-news/k...,Kiss are the latest artists to sell off their ...,473
...,...,...,...,...,...,...,...
85,The Times of India,Singer Abhijeet Bhattacharya targets Bollywood...,2024-04-04 05:28:19+00:00,None,https://timesofindia.indiatimes.com/videos/eti...,"Apr 04, 2024, 10:58AM IST Source: etimes.in ...",230
86,Bollywood Hungama,BREAKING: Bade Miyan Chote Miyan passed with a...,2024-04-04 04:13:15+00:00,Bollywood Hungama,https://www.bollywoodhungama.com/news/bollywoo...,"Yesterday, Bollywood Hungama reported that Aja...",365
87,The Times of India,Salman Khan is not filing a case against comed...,2024-04-04 03:28:00+00:00,TOI Entertainment Desk,https://timesofindia.indiatimes.com/entertainm...,The alleged defamation incident involving Salm...,165
88,Deadline,Gillian Anderson Won’t Scuttle The Idea Of Ret...,2024-04-04 03:23:00+00:00,Bruce Haring,http://deadline.com/2024/04/gillian-anderson-w...,"On Wednesday’s episode of the Today show, Gill...",225


In [2]:
day_before_

,source,title,publishedAt,author,url
0,Entertainment Weekly,9-1-1's Oliver Stark breaks down Buck's kiss w...,2024-04-05 01:01:13+00:00,https://www.facebook.com/entertainmentweekly,https://ew.com/911-buck-tommy-kiss-bisexual-ol...
1,Fox News,"Madonna fires back at concert lawsuit, claims ...",2024-04-05 00:58:00+00:00,Brie Stimson,https://www.foxnews.com/entertainment/madonna-...
2,Vulture,"'Ripley' Recap, Episode 6: Some Heavy Instrume...",2024-04-05 00:48:06+00:00,Rafaela Bassili,http://www.vulture.com/article/ripley-recap-ep...
3,Fox Business,Rebel Wilson claims agency 'liked me fat' afte...,2024-04-05 00:42:00+00:00,Janelle Ash,https://www.foxbusiness.com/entertainment/rebe...
4,Rolling Stone,"Kiss Sell Music Catalog, Publishing, and Image...",2024-04-05 00:35:00+00:00,Daniel Kreps,http://www.rollingstone.com/music/music-news/k...
...,...,...,...,...,...
85,The Times of India,Singer Abhijeet Bhattacharya targets Bollywood...,2024-04-04 05:28:19+00:00,None,https://timesofindia.indiatimes.com/videos/eti...
86,Bollywood Hungama,BREAKING: Bade Miyan Chote Miyan passed with a...,2024-04-04 04:13:15+00:00,Bollywood Hungama,https://www.bollywoodhungama.com/news/bollywoo...
87,The Times of India,Salman Khan is not filing a case against comed...,2024-04-04 03:28:00+00:00,TOI Entertainment Desk,https://timesofindia.indiatimes.com/entertainm...
88,Deadline,Gillian Anderson Won’t Scuttle The Idea Of Ret...,2024-04-04 03:23:00+00:00,Bruce Haring,http://deadline.com/2024/04/gillian-anderson-w...
